#README.txt

Drag and drop a ZIPPED version of the folder `/data` from GitHub repo into Colab.

< - - - - -

In [1]:
# then run this and refresh directory...

# unzip datasets
!unzip data.zip

Archive:  data.zip
   creating: data/job-postings/
  inflating: data/job-postings/Data_Job_NY.csv  
  inflating: data/job-postings/Data_Job_SF.csv  
  inflating: data/job-postings/Data_Job_TX.csv  
  inflating: data/job-postings/Data_Job_WA.csv  
  inflating: data/README.txt         
  inflating: data/resume.pdf         
   creating: data/resumes/
  inflating: data/resumes/kaggleResumes.csv  




---


# API

In [96]:
!pip install config

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [97]:
import requests
import os
import config
# API Docs found here: https://developer.usajobs.gov/Tutorials/Search-Jobs

In [98]:
config.US_JOBS_API_KEY = "xVc0TZiLfhcr17ci7Ngk6bLAetdRVFgntm2pZgWNtww="
config.EMAIL_ADDRESS = "gjacobthomas@gmail.com"

Note: I'm assuming this API has some limiter on it so we don't want to lose access. -tyler

In [99]:
# more parameters are found here: https://developer.usajobs.gov/API-Reference/GET-api-Search
def queryJobsAPI(keyword, location):
  host = 'data.usajobs.gov' 
  # add these values in the config.py file
  userAgent = config.EMAIL_ADDRESS
  authKey = config.US_JOBS_API_KEY

  base_url = "https://data.usajobs.gov/api/search"

  parameters = {
      "Keyword": keyword,
      "LocationName": location
  }

  headers = {
      "Host": host,          
      "User-Agent": userAgent,          
      "Authorization-Key": authKey  
  }

  resp = requests.request("GET", base_url,headers=headers, params=parameters)
  result = resp.json()['SearchResult']['SearchResultItems']

  # get Job Title 
  print(result[1]['MatchedObjectDescriptor']['PositionTitle'])
  # get Job Summary
  print(result[1]['MatchedObjectDescriptor']['UserArea']['Details']['JobSummary'])
  return result




---


# KeyBERT Extraction Function

In [6]:
!pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.0 MB/s 
     |████████████████████████████████| 237 kB 39.5 MB/s 
     |████████████████████████████████| 51 kB 5.8 MB/s 
     |████████████████████████████████| 5.8 MB 58.2 MB/s 
     |████████████████████████████████| 1.3 MB 67.4 MB/s 
     |████████████████████████████████| 182 kB 69.8 MB/s 
     |████████████████████████████████| 7.6 MB 35.5 MB/s 
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23800 sha256=7d03e4ee3a19d7307c78831aa1b1ed4b15cd46b88f3f8772a7fa19ca7d1b2208
  Stored in directory: /root/.cache/pip/wheels/6c/bc/8b/a51bee77aec33895e6c8c236144b4cc10875659c4d2c80f070
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=356242bd0129c9050436fb16cce045bac8fc828626bb85f0959fa9d37eae7279
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88

In [100]:
# Imports
from keybert import KeyBERT # pip install keybert (give it a minute...)
from statistics import mean

'''
/*---------------------------------------------------------------------
 |  Method: extractKeywordsBERT
 |
 |  Purpose: Uses the KeyBert Keyword Extraction Tool to extract
 |           and return keywords from a given corpus. 
 |      
 |  Author: Tyler Parks
 |  Created On: 10/30/22
 |
 |  Parameters:
 |      normalized_corpus -- A single string containing all text of the
 |                           normalized corpus.
 |
 |  Returns: 
 |      keywords -- List of collected keywords
 |      scores -- List of those keyword's scores
 |
 |  References: https://maartengr.github.io/KeyBERT/#usage
 |
 *-------------------------------------------------------------------*/
''' 
def extractKeywordsBERT(normalized_corpus):   
    print('---KeyBert Extraction---')
    print('------------------------\n')

    # init. language model 
    language_model = KeyBERT(model = 'all-mpnet-base-v2')

    # extract those keywords!
    data = language_model.extract_keywords( normalized_corpus, 
                                            keyphrase_ngram_range=(1, 3), 
                                            stop_words='english',
                                            use_maxsum=False, 
                                            use_mmr=True,
                                            diversity=0.7,
                                            nr_candidates=20, 
                                            top_n=15
                                        )

    # zip the lists
    zipped = list(map(list, zip(*data)))
    keywords = zipped[0]
    scores = zipped[1]

    print('-Skill-'.ljust(40), '-Score-')
    for i, value in enumerate(keywords):
        print(value.ljust(40), scores[i])
    print()

    # print("type: "+str(type(scores))+"length: "+ str(len(scores)))

    avg = mean(scores[:14])
    print("Score Average: " + str(avg))

    return keywords, scores, avg



---


# CSO-Classifier Extraction Function

In [ ]:
!python -m spacy download en_core_web_sm
!pip install cso-classifier

**Don't restart runtime if the terminal says so! Keep going.**

In [ ]:
import nltk
nltk.download('stopwords')
# update spacy
!pip install -U spacy
import spacy
from cso_classifier import CSOClassifier      # import classifier tool

In [ ]:
# update the most recent model
CSOClassifier.update() 

# define the model object
CSO_Extractor = CSOClassifier(modules = "both", enhancement = "first", explanation = False)

In [101]:
def extractKeywordsCSO(normalized_corpus):

  # run the extraction
  result = CSO_Extractor.run(normalized_corpus)

  print('\n-----CSO Extraction-----')
  print('------------------------\n')
  
  for keyword in result['union']:
    print(keyword)

  return result['union']

# **YAKE Extractor**

In [102]:
!pip3 install yake
import yake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [103]:
def extractKeywordsYAKE(normalized_corpus):
    print()
    print('\n---YAKE Extraction---')
    print('------------------------\n')

    language = "en"
    max_ngram_size = 3
    deduplication_threshold = 0.9
    deduplication_algo = 'seqm'
    windowSize = 1
    numOfKeywords = 20

    kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
    keywords = kw_extractor.extract_keywords(normalized_corpus)

    zipped = list(map(list, zip(*keywords)))
    keywords = zipped[0]
    scores = zipped[1]

    print('-Skill-'.ljust(40), '-Score-')
    for i, value in enumerate(keywords):
        print(value.ljust(40), scores[i])

    avg = mean(scores[:19])
    print("Score Average: " + str(avg)+ "\n")
    
    return keywords, scores, avg

# **TextRank Extractor**

In [93]:
!pip install summa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 2.1 MB/s 
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54410 sha256=1c58eef039ccb6b6597897017e208866675a77947119c97e3ae3506efb821047
  Stored in directory: /root/.cache/pip/wheels/fd/6a/dd/209eb19d5f2266b9cfd06827539bf70435b0ad5fe8244e52d3
Successfully built summa


In [94]:
from summa import keywords

In [95]:
def extractKeywordsTextRank(normalized_corpus):
  print('\n')
  print('---TextRank Extraction---')
  print('------------------------\n')

  #extract
  TR_keywords = keywords.keywords(normalized_corpus, scores=True)

  #zip into list
  zipped = list(map(list, zip(*TR_keywords)))
  TR_keywords = zipped[0]
  scores = zipped[1]
  
  #print(TR_keywords[0:20])
  print('-Skill-'.ljust(40), '-Score-')
  for i, value in enumerate(TR_keywords[0:19]):
    print(value.ljust(40), scores[i])
    
  avg = mean(scores[:19])
  print("\nScore Average: " + str(avg))
  
  return TR_keywords, scores, avg

# **Skill Matching**

In [46]:
!pip install gensim
!pip install jieba

from functools import reduce
import gensim
import nltk
from nltk.tokenize import word_tokenize
import jieba
nltk.download('punkt')
import re
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [65]:
import textwrap
wrapper = textwrap.TextWrapper(width=175)  # wrapper options

In [79]:
class SkillsMatching:
  def __init__(self, all_job_postings_keywords, all_jp_titles, all_resumes_keywords):
      self.all_job_postings_keywords = all_job_postings_keywords
      self.all_jp_titles = all_jp_titles
      self.all_resumes_keywords = all_resumes_keywords
      

  # output JSON with matching job postings for each resume
  def skill_match(self):
      results = []
      
      for resume_json in self.all_resumes_keywords:
          resume_arr = []
          for key in resume_json:
            # loop through the keys = every classifier 
            classifier_json = {key: {"matching_job": "",
                      "job_title": ""}}
            resume_keywords = resume_json[key]
            scores = self.sim_score(self.all_job_postings_keywords, resume_keywords)

            # grab largest score
            largestScore = max(scores)

            # grab the index of the largest score in the arr
            max_idx = np.argmax(scores)

            classifier_json[key]['matching_job'] = self.all_job_postings_keywords[max_idx]
            classifier_json[key]['job_title'] = self.all_jp_titles[max_idx]
            classifier_json[key]['score'] = largestScore

            resume_arr.append(classifier_json)
          results.append(resume_arr)
      return results


  def split_and_join_arr(self, arr):
    new_arr = []
    for w in arr:
      word_arr = re.split('\W+', w.lower())
      new_arr = new_arr + word_arr
    # print(new_arr)
    return new_arr
  
  def sim_score(self, docs, keywords):
    keywords = self.split_and_join_arr(keywords)
    gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in docs]
    dictionary = gensim.corpora.Dictionary(gen_docs)
    corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
    tf_idf = gensim.models.TfidfModel(corpus)
    sims = gensim.similarities.Similarity('/usr/workdir',tf_idf[corpus],
                                      num_features=len(dictionary))

    query_doc_bow = dictionary.doc2bow(keywords)
    query_doc_tf_idf = tf_idf[query_doc_bow]
    # print(sims[query_doc_tf_idf])
    return sims[query_doc_tf_idf]

  def print_results(self, match_job_postings):

    # init average scores
    avgScoreBERT = 0
    avgScoreCSO = 0
    avgScoreYAKE = 0

    # for each resume
    for i, resume in enumerate(match_job_postings):
      print('\n------------------------')
      print("Matching Job Posting for Resume %s" % (i + 1))
      for classifier_obj in resume:
        classifier_type = list(classifier_obj.keys())[0]

        # if BERT
        if classifier_type == 'keywordsBERT':
          avgScoreBERT += round(classifier_obj[classifier_type]['score'] * 100, 2)

        # else if CSO
        elif classifier_type == 'keywordsCSO':
          avgScoreCSO += round(classifier_obj[classifier_type]['score'] * 100, 2)

        # else YAKE
        else:
          avgScoreYAKE += round(classifier_obj[classifier_type]['score'] * 100, 2)

        print()
        print('Skill Extracted Using:', classifier_type)
        print('Matched JP Title:', classifier_obj[classifier_type]['job_title'])
        print('With a score of:', round(classifier_obj[classifier_type]['score'] * 100, 2), '%')

        # Wrap this text.
        word_list = wrapper.wrap(text = 
              'Matched JP Description: ' + classifier_obj[classifier_type]['matching_job'])

        # Print each line.
        for element in word_list:
            print(element)

    avgScoreBERT /= len(match_job_postings)
    avgScoreCSO /= len(match_job_postings)
    avgScoreYAKE /= len(match_job_postings)

    print()
    print('---POST PROCESSING ANALYSIS----')
    print('-------------------------------')
    print()
    print('Average Matching Scores for Skill Extraction Method')
    print('BERT: ', round(avgScoreBERT, 2), '%')
    print('CSO : ', round(avgScoreCSO, 2), '%')
    print('YAKE: ', round(avgScoreYAKE, 2), '%')



---


#Driver Code

In [104]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 5.0 MB/s 
     |████████████████████████████████| 110 kB 55.3 MB/s 


In [105]:
# Imports
import pandas as pd     # pip install pandas. usage: loading data from csv files into dataframes
import nltk
from nltk.tokenize import word_tokenize  
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.utils import shuffle
import re
import contractions
import spacy
from numpy.lib.npyio import savez_compressed
from array import *
nltk.download("all")
nltk.download('stopwords')
from nltk.corpus import stopwords
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

In [107]:
### Helper Functions

# Function to retrieve text data.
# (either 1 or more job postings or resumes)?
def getFileData(filename, dir):
    return pd.read_csv('data/' + dir + '/' + filename)

# Function to normalize text data. 
# (some skill extraction tools will normalize text for us; however, if not, this function is here)
# includes removing stopwords, punctuation, dates, links, etc...
def normalizeCorpus(corpus):

    # import nltk for stopwords
    stop_words = set(stopwords.words('english'))

    # convert to lower case
    lower_corpus = corpus.lower()

    # remove numbers
    no_number_corpus = re.sub(r'\d+','',lower_corpus)

    # remove all punctuation except words and space
    no_punc_corpus = re.sub(r'[^\w\s]','', no_number_corpus)

    # remove white spaces
    no_wspace_corpus = no_punc_corpus.strip()
    no_wspace_corpus

    # convert string to list of words
    lst_corpus = [no_wspace_corpus][0].split()
    print(lst_corpus)

    # remove stopwords
    no_stpwords_corpus=""
    for i in lst_corpus:
	    if not i in stop_words:
	      no_stpwords_corpus += i+' '
		
    # removing last space
    no_stpwords_corpus = no_stpwords_corpus[:-1]

    # output
    print(no_stpwords_corpus)

    return no_stpwords_corpus

    # need to fix this 
    # nltk_tokenList = word_tokenize(corpus)
    # lemmatizer = WordNetLemmatizer()
    # nltk_lemmaList = []
    # for word in nltk_tokenList:
    #     nltk_lemmaList.append(lemmatizer.lemmatize(word))

    # normalized_corpus = []  
    # nltk_stop_words = set(stopwords.words("english"))
    # for w in nltk_lemmaList:  
    #     if w not in nltk_stop_words:  
    #         normalized_corpus.append(w)

    # punctuation = ";:.,?!"
    # for word in normalized_corpus:
    #     if word in punctuation:
    #         normalized_corpus.remove(word)

    #still need to add dates and links
    # return normalized_corpus

def similiartychecker(tokenA, tokenB):

    A = nlp(tokenA)
    B = nlp(tokenB)
  
    score = A.similarity(B)
  
    return score

# Function to extract skill words from a given corpus.
# ideally, this function will output a set of skills extracted from the corpus
def extractSkills(corpus):

    # run KeyBert Extraction
    keywordsBERT, scoresBERT, avgBERT = extractKeywordsBERT(corpus)

    # run CSO Classifier Extraction
    keywordsCSO = extractKeywordsCSO(corpus)
    
    # extraction method 2
    keywordsYAKE, scoresYAKE, avgYAKE = extractKeywordsYAKE(corpus)

    # extraction method 3
    keywordsTextRank, scoresTextRank, avgTextRank = extractKeywordsTextRank(corpus)

    # return keywordsBERT, keywordsYAKE[0], keywordsTextRank[0]
    # return keywordsBERT, keywordsCSO, keywordsYAKE, keywordsTextRank,
    return {"keywordsBERT": keywordsBERT,
            "keywordsCSO": keywordsCSO,
            "keywordsYAKE": keywordsYAKE}
            # "keywordsTextRAnk": keywordsTextRank}

def extractSkillsforcomp(corpus):

    # run KeyBert Extraction
    keywordsBERT, scoresBERT, avgBERT = extractKeywordsBERT(corpus)

    # run CSO Classifier Extraction
    keywordsCSO = extractKeywordsCSO(corpus)
    
    # extraction method 2
    keywordsYAKE, scoresYAKE, avgYAKE = extractKeywordsYAKE(corpus)

    # extraction method 3
    keywordsTextRank, scoresTextRank, avgTextRank = extractKeywordsTextRank(corpus)

    # return keywordsBERT, keywordsYAKE[0], keywordsTextRank[0]
    return keywordsBERT, keywordsCSO, keywordsYAKE, keywordsTextRank

### Driver Code
if __name__ == '__main__':

    # fetch the data
    jpCorpus = []
    jpTitles = []

    # --- FETCH MORE DATA VIA API ---
    job_posting_obj = queryJobsAPI("Data Scientist", "Washington, DC")
    for i in range(len(job_posting_obj)):
      jpCorpus.append(job_posting_obj[i]['MatchedObjectDescriptor']['UserArea']['Details']['JobSummary'])
      jpTitles.append(job_posting_obj[i]['MatchedObjectDescriptor']['PositionTitle'])

    # --- FETCH DATA FROM /CONTENT/DATA/..


    # collect all Job Postings
    dfNY = getFileData('Data_Job_NY.csv', 'job-postings')
    dfSF = getFileData('Data_Job_SF.csv', 'job-postings')
    dfTX = getFileData('Data_Job_TX.csv', 'job-postings')
    dfWA = getFileData('Data_Job_WA.csv', 'job-postings')

    frames = [dfNY, dfSF, dfTX, dfWA] # build frame
    job_posting_dataframe = pd.concat(frames) # concat frames
    job_posting_dataframe = shuffle(job_posting_dataframe)  # shuffle df

    resume_dataframe      = getFileData('kaggleResumes.csv', 'resumes')

    jpCorpus += list(job_posting_dataframe['Job_Desc'])
    jpTitles += list(job_posting_dataframe['Job_title'])
    #---------------

    """
    # print the dataframes
    #print('DataFrame of Job Postings:')
    #print(job_posting_dataframe)    
    #print()

    #print('DataFrame of Resumes:')
    #print(resume_dataframe)
    #print()
    #----------------------
    """

    # fetch resumes by themselves
    rCorpus  = list(resume_dataframe['Resume'])
    #----------------------

    # Number of resume samples to view
    NUM_SAMPLES = 1
    # number of job postings to view
    NUM_JPS = 1

    print('\nCurrently Avaliable Job Postings: ', len(jpCorpus))
    print()

    # for each JOB POSTING from the corpus
    i = 0
    all_job_postings = []
    for posting in jpCorpus:
          
        # Wrap this text.
        word_list = wrapper.wrap(text=posting)
    
        print('Job Posting #', i+1)

        # Print each line.
        for element in word_list:
            print(element)    

        print()

        text = normalizeCorpus(posting)
        job_posting_json = extractSkills(text)
        all_job_postings.append(text)

        #for similiarity referencer:
        jobKeyBERTsimComp, jobCSOsimComp, jobYAKEsimComp, jobTextRanksimComp = extractSkillsforcomp(text)

        # print lines, we are done with this posting
        print('------------------------\n')

        # break, after X postings
        if i > NUM_JPS - 1:
          break

        i += 1

    #---------------------- 

    # for each RESUME from the corpus
    i = 0
    all_resumes_keywords = []
    for resume in rCorpus:

        # Wrap this text.
        word_list = wrapper.wrap(text = resume)
    
        print('Resume #', i+1)

        # Print each line.
        for element in word_list:
            print(element)    

        print()

        text = normalizeCorpus(resume)
        resume_json = extractSkills(text)
        all_resumes_keywords.append(resume_json)

        #for similiarity referencer:
        resKeyBERTsimComp, resCSOsimComp, resYAKEsimComp, resTextRanksimComp = extractSkillsforcomp(text)

        # print lines, we are done with this resume
        print('------------------------\n')

        # break, after X resumes
        if i > NUM_SAMPLES - 1:
            break

        i += 1
    #---------------------- 

    '''
    ----------------------------------------------------------------------|
    |coss reference similiary of resume and job posting extracted keywords|
    |-addtion by Tyrell Richardson                                        |
    |                                                                     |
    |---------------------------------------------------------------------|
    '''

    keyBERTSimiliarity = np.zeros(14)

    for k in range (0,14):
        keyBERTSimiliarity[k] = similiartychecker(jobKeyBERTsimComp[k], resKeyBERTsimComp[k])

    print('KeyBERT match Similarity Mean: '+str(np.mean(keyBERTSimiliarity)))

    #-----------------------------------------------------------------------------

    CSOSimilarity = np.zeros(len(min(jobCSOsimComp, resCSOsimComp)))

    for h in range (0,3):
      CSOSimilarity[h] = similiartychecker(jobCSOsimComp[h],resCSOsimComp[h])

    print('CSO match Similiarity Mean: '+str(np.mean(CSOSimilarity)))

    #-----------------------------------------------------------------------------

    YakeSimilarity = np.zeros(len(min(jobYAKEsimComp, resYAKEsimComp)))
    for i in range (0,19):
      YakeSimilarity[i] = similiartychecker(jobYAKEsimComp[i],resYAKEsimComp[i])

    print('Yake match Similiarity Mean: '+str(np.mean(YakeSimilarity)))

    #-----------------------------------------------------------------------------

    TextRankSimiliarity = np.zeros(5)
    for j in range (0, 5):
      TextRankSimiliarity[j] = similiartychecker(jobTextRanksimComp[j], resTextRanksimComp[j])

    print('TextRank match Similarity Mean: '+str(np.mean(TextRankSimiliarity)))



    
# keep going!

# end of driver code
#---------------

Data Scientist
This position is located in the Department of Health & Human Services (HHS), Centers for Medicare & Medicaid Services (CMS), Office of Burden Reduction and Health Informatics (OBRHI), Emerging Innovations Group . As a Data Scientist, GS-1560-13, you will design, develop, and implement the analytical, statistical, and programming mechanisms necessary to collect, organize, analyze, and interpret unique and highly specialized data sets.

Currently Avaliable Job Postings:  3349

Job Posting # 1
As a Data Scientist, you will provide data management and analytical support to the Technology Branch Office, Office of Public Trust, specific to the body worn camera and
electronic records management systems. This office was created to strengthen public trust in the National Park Service's law enforcement programs through the transparency,
availability, and accessibility of information. This is a fully remote position.

['as', 'a', 'data', 'scientist', 'you', 'will', 'provide', 'data

# Resume and Job Matching

In [108]:
# Resume and Job Posting Matching
matching = SkillsMatching(all_job_postings, jpTitles, all_resumes_keywords)
match_job_postings = matching.skill_match()
matching.print_results(match_job_postings)


------------------------
Matching Job Posting for Resume 1

Skill Extracted Using: keywordsBERT
Matched JP Title: Data Scientist
With a score of: 23.57 %
Matched JP Description: data scientist provide data management analytical support technology branch office office public trust specific body worn camera electronic records
management systems office created strengthen public trust national park services law enforcement programs transparency availability accessibility information fully remote
position

Skill Extracted Using: keywordsCSO
Matched JP Title: Data Scientist
With a score of: 16.9 %
Matched JP Description: position located department health human services hhs centers medicare medicaid services cms office burden reduction health informatics obrhi emerging
innovations group data scientist gs design develop implement analytical statistical programming mechanisms necessary collect organize analyze interpret unique highly
specialized data sets

Skill Extracted Using: keywordsYAKE


# Resume Upload

Drag and Drop a Data Scientist resum into Colab 

Make sure the file is named **resume.pdf**

There is a dummy resume located in the `/data` directory.



< - - - - - - -

In [67]:
!pip install pdfminer3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [109]:
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io
import sys
from contextlib import redirect_stdout
from io import StringIO 

In [110]:
resource_manager = PDFResourceManager()
fake_file_handle = io.StringIO()
converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
page_interpreter = PDFPageInterpreter(resource_manager, converter)

with open('/content/data/resume.pdf', 'rb') as fh:

    for page in PDFPage.get_pages(fh,
                                  caching=True,
                                  check_extractable=True):
        page_interpreter.process_page(page)

    text = fake_file_handle.getvalue()

# close open handles
converter.close()
fake_file_handle.close()

# normalize and extract skills 
text = normalizeCorpus(text)
resume_kw = extractSkills(text)

# Matching
matching = SkillsMatching(all_job_postings, jpTitles, [resume_kw])
match_job_postings = matching.skill_match()
matching.print_results(match_job_postings)

['juan', 'jose', 'carin', 'data', 'scientist', 'professional', 'profile', 'mountain', 'view', 'ca', 'juanjosecaringmailcom', 'juanjocaringithubio', 'linkedincominjuanjosecarin', 'passionate', 'about', 'data', 'analysis', 'and', 'experiments', 'mainly', 'focused', 'on', 'user', 'behavior', 'experience', 'and', 'engagement', 'with', 'a', 'solid', 'background', 'in', 'data', 'science', 'and', 'statistics', 'and', 'extensive', 'experience', 'using', 'data', 'insights', 'to', 'drive', 'business', 'growth', 'education', 'relevant', 'courses', 'university', 'of', 'california', 'berkeley', 'machine', 'learning', 'machine', 'learning', 'at', 'scale', 'storing', 'and', 'retrieving', 'data', 'master', 'of', 'information', 'and', 'data', 'science', 'field', 'experiments', 'applied', 'regression', 'and', 'time', 'series', 'data', 'visualization', 'and', 'communication', 'gpa', 'analysis', 'research', 'design', 'and', 'applications', 'for', 'exploring', 'and', 'analyzing', 'data', 'data', 'analysis'